# Autocombat  5 way harmonized datasets

Note this must be run in the `comscan6` environment

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har
import cvasl.vendor.comscan.neurocombat as autocombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
# Datasets for this work
Insight46_path = '../our_datasets/Insight46/'
MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
SABRE_path = '../our_datasets/SABRE/'
EDIS_path = '../our_datasets/EDIS/'
HELIUS_path = '../our_datasets/HELIUS/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
SABRE_file = os.path.join(SABRE_path, file_name)
EDIS_file = os.path.join(EDIS_path, file_name)
Insight46_file = os.path.join(Insight46_path, file_name)
HELIUS_file = os.path.join(HELIUS_path, file_name)

TOP = pd.read_csv(TOP_file,index_col=0)
MRI = pd.read_csv(MRI_file,index_col=0)
Insight46 = pd.read_csv(Insight46_file,index_col=0)
SABRE = pd.read_csv(SABRE_file,index_col=0)
EDIS = pd.read_csv(EDIS_file,index_col=0)
HELIUS = pd.read_csv(HELIUS_file,index_col=0)

### re-assign sex where neccesary

In [ ]:
sex_mapping = {1:0,2:1}
Insight46 = Insight46.assign(Sex = Insight46.Sex.map(sex_mapping))
Insight46.head(3)

In [ ]:
EDIS = EDIS.assign(Sex = EDIS.Sex.map(sex_mapping))
EDIS.head(3)

In [ ]:
SABRE = SABRE.assign(Sex = SABRE.Sex.map(sex_mapping))
SABRE.head(3)

In [ ]:
### drop ID columns

In [ ]:
TOP = TOP.drop([ 'ID'], axis= 1)
TOP.head(3)

In [ ]:
MRI = MRI.drop(['ID'],axis = 1)
MRI.tail(5)

In [ ]:
EDIS = EDIS.drop(['ID'],axis = 1)
EDIS.tail(5)

In [ ]:
SABRE = SABRE.drop(['ID'],axis = 1)
SABRE.tail(5)

In [ ]:
HELIUS = HELIUS.drop(['ID'],axis = 1)
HELIUS.tail(5)

In [ ]:
Insight46 = Insight46.drop(['ID'],axis = 1)
Insight46.tail(5)

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI['Site'] = 0
TOPMRI.head(3) 

In [ ]:
datasets = [TOPMRI, HELIUS, SABRE, EDIS, Insight46]
for everyone in datasets:
    everyone.columns =  everyone.columns.str.lower()
TOPMRI.head(3)

In [ ]:
TOPMRI['site'] = 0
EDIS['site'] = 1
HELIUS['site'] = 2
SABRE['site'] = 3
Insight46['site'] = 4

In [ ]:
FIVEDATA = pd.concat(datasets)


In [ ]:
FIVEDATA['decade']=(FIVEDATA['age']/10).round()
#FIVEDATA['decade']

In [ ]:
FIVEDATA = FIVEDATA.sort_values(by='age')
FIVEDATA.reset_index(inplace=True)
FIVEDATA['fine_grain'] = FIVEDATA['age'].rolling(2).sum()/2
FIVEDATA

In [ ]:
FIVEDATA.fine_grain

In [ ]:
FIVEDATA[:].fine_grain.iloc[::2] = FIVEDATA[:].fine_grain.iloc[1::2]
FIVEDATA['fine_grain']

In [ ]:
#FIVEDATA['fine_grain'][-1] = FIVEDATA['fine_grain'][-2]

In [ ]:
FIVEDATA['decade'].unique()

In [ ]:
FIVEDATA.columns

In [ ]:
combat = autocombat.Combat(
    features=[ 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
        'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
        'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['decade'],)

fg_combat = autocombat.Combat(
    features=[ 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['fine_grain'],)


In [ ]:
print(combat.fit(FIVEDATA))
print(fg_combat.fit(FIVEDATA))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)

In [ ]:
transformed_FIVEDATA= combat.transform(FIVEDATA)
transformed_FIVEDATA.head(3)

In [ ]:
fg_transformed_FIVEDATA= fg_combat.transform(FIVEDATA)
fg_transformed_FIVEDATA.head(3)

In [ ]:
FIVEDATA.head(3)

In [ ]:
TOPMRI_transformed = transformed_FIVEDATA[ transformed_FIVEDATA['site']==0]
TOPMRI_transformed = TOPMRI_transformed.drop(['site', 'decade','fine_grain', 'index'], axis=1)
TOPMRI_transformed.head(3)

In [ ]:
fg_TOPMRI_transformed = fg_transformed_FIVEDATA[fg_transformed_FIVEDATA['site']==0]
fg_TOPMRI_transformed = fg_TOPMRI_transformed .drop(['site', 'decade','fine_grain', 'index'], axis=1)
fg_TOPMRI_transformed.head(3)

In [ ]:
EDIS_transformed = transformed_FIVEDATA[ transformed_FIVEDATA['site']==1]
EDIS_transformed = EDIS_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
EDIS_transformed.head(3)

In [ ]:
HELIUS_transformed = transformed_FIVEDATA[ transformed_FIVEDATA['site']==2]
HELIUS_transformed = HELIUS_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
HELIUS_transformed.head(3)

In [ ]:
SABRE_transformed = transformed_FIVEDATA[ transformed_FIVEDATA['site']==3]
SABRE_transformed = SABRE_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
SABRE_transformed.head(3)

In [ ]:
Insight_transformed = transformed_FIVEDATA[ transformed_FIVEDATA['site']==4]
Insight_transformed = Insight_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
Insight_transformed.head(3)

In [ ]:
fg_EDIS_transformed = fg_transformed_FIVEDATA[fg_transformed_FIVEDATA['site']==1]
fg_EDIS_transformed = fg_EDIS_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
fg_EDIS_transformed.head(3)

In [ ]:
fg_HELIUS_transformed = fg_transformed_FIVEDATA[fg_transformed_FIVEDATA['site']==2]
fg_HELIUS_transformed = fg_HELIUS_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
fg_HELIUS_transformed.head(3)

In [ ]:
fg_SABRE_transformed = fg_transformed_FIVEDATA[fg_transformed_FIVEDATA['site']==3]
fg_SABRE_transformed = fg_SABRE_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
fg_SABRE_transformed.head(3)

In [ ]:
fg_Insight_transformed = fg_transformed_FIVEDATA[fg_transformed_FIVEDATA['site']==4]
fg_Insight_transformed = fg_Insight_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
fg_Insight_transformed.head(3)

In [ ]:
TOPMRI_transformed.to_csv('harm_results/autocombat/5autocom5_harm_topmri_v_e.csv')
EDIS_transformed.to_csv('harm_results/autocombat/5autocom5_harm_EDIS.csv')
HELIUS_transformed.to_csv('harm_results/autocombat/5autocom5_harm_HELIUS.csv')
SABRE_transformed.to_csv('harm_results/autocombat/5autocom5_harm_SABRE.csv')
Insight_transformed.to_csv('harm_results/autocombat/5autocom5_harm_Insight.csv')

fg_TOPMRI_transformed.to_csv('harm_results/autocombat/5fg_autocom5_harm_topmri_v_e.csv')
fg_EDIS_transformed.to_csv('harm_results/autocombat/5fg_autocom5_harm_EDIS1.csv')
fg_HELIUS_transformed.to_csv('harm_results/autocombat/5fg_autocom5_harm_HELIUS.csv')
fg_SABRE_transformed.to_csv('harm_results/autocombat/5fg_autocom5_harm_SABRE.csv')
fg_Insight_transformed.to_csv('harm_results/autocombat/5fg_autocom5_harm_Insight.csv')